<a href="https://colab.research.google.com/github/charlie-aashutosh/artificial_intelligence/blob/main/Bidirectional_UCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def bidirectional_ucs(graph, start, goal, heuristic=euclidean_dist_heuristic):
  
    if start == goal:
	return []

    pathCost = 0
    mu = float("inf")
    interNode = None

    srcFrontier = PriorityQueue()
    srcExplored = []
    srcPathMap = {}
    srcG = {}
    srcFrontier.append((pathCost,start))
    srcG[start] = pathCost

    tgtFrontier = PriorityQueue()
    tgtExplored = []
    tgtPathMap = {}
    tgtG = {}
    tgtFrontier.append((pathCost,goal))
    tgtG[goal] = pathCost

    while srcFrontier.size() != 0 and tgtFrontier.size() != 0:
	sCost = srcFrontier.top()	
	gCost = tgtFrontier.top()


	if sCost[0] + gCost[0] >= mu:
		fPath = getFPath(srcPathMap, start, interNode)
		rPath = getFPath(tgtPathMap, goal, interNode)
		finalPath = fPath
		rPath = rPath[:-1]
		for rev in reversed(rPath):
			finalPath.append(rev)
		print 'finalPath - ', finalPath
		return finalPath

	# Forward Search
	fPath = srcFrontier.pop()
	fNode = fPath[1]
	fCost = fPath[0]
	srcExplored.append(fNode)
	for fNeigh in graph.neighbors(fNode):
		stepCost = graph[fNode][fNeigh]['weight']
		pathCost = fCost + stepCost


		if fNeigh in tgtExplored:
			total = pathCost + tgtG[fNeigh] 
			if total < mu:
				mu = total				
				interNode = fNeigh


		if srcFrontier.__contains__(fNeigh) is True:
			it = srcFrontier.__iter__()
			for nodeItr in it:
				if nodeItr[1] == fNeigh and nodeItr[0] > pathCost:
					srcFrontier.remove(nodeItr)
		if fNeigh not in srcExplored \
		and srcFrontier.__contains__(fNeigh) is False:
			srcPathMap[fNeigh] = fNode
			srcG[fNeigh] = pathCost
			srcFrontier.append((pathCost,fNeigh))

	# Reverse Search
	rPath = tgtFrontier.pop()
	rNode = rPath[1]
	rCost = rPath[0]
	tgtExplored.append(rNode)
	for rNeigh in graph.neighbors(rNode):
		stepCost = graph[rNode][rNeigh]['weight']
		pathCost = rCost + stepCost


		if rNeigh in srcExplored:
			total = pathCost + srcG[rNeigh]
			if total < mu:
				mu = total
				interNode = rNeigh


		if tgtFrontier.__contains__(rNeigh) is True:
			it = tgtFrontier.__iter__()
			for nodeItr in it:
				if nodeItr[1] == rNeigh and nodeItr[0] > pathCost:
					tgtFrontier.remove(nodeItr)
		if rNeigh not in tgtExplored \
		and tgtFrontier.__contains__(rNeigh) is False:
			tgtPathMap[rNeigh] = rNode
			tgtG[rNeigh] = pathCost
			tgtFrontier.append((pathCost,rNeigh))

def pf(start,goal,node,heuristic,graph):
	pi_f = heuristic(graph, node, goal)
	pi_r = heuristic(graph, start, node)
	return 0.5*(pi_f - pi_r)